In [1]:
!conda env list
!python -V
import sys
print(sys.executable)

# conda environments:
#
base                     /Applications/anaconda3
phd                   *  /Applications/anaconda3/envs/phd

Python 3.10.0
/Applications/anaconda3/envs/phd/bin/python


In [2]:
# DATA ANALYSIS
import numpy as np
import pandas as pd
import random as rand

# PLOTS
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# OTHERS
import math
import statistics as stats
import time as t
#import pickle
import functools as fc # contains reduce
import copy

# DEBUGGER
from IPython.core.debugger import set_trace
# For executing line by line use n and 
# for step into a function use s and 
# to exit from debugging prompt use c.
# REFACTOR: uninstall jupyter packages that I installed before, not using PyCharm built in debugger


# import color blind colors
colorblind = ['#377eb8', '#ff7f00', '#4daf4a',
              '#f781bf', '#a65628', '#984ea3',
              '#999999', '#e41a1c', '#dede00']

# Library <a class="anchor" id="1"></a>

In [3]:
# Utility functions
%load utils.py
import utils

# Belief propagation <a class="anchor" id="3.2"></a>
Also known as message passing

In [4]:
# Lung Function Model
class lungModel:
    
    # Names
    """
    i: inflammation
    bl: bacterial load
    w: wellness
    """
    # Marginal distributions
    marginal_i = {
        "absent": 0.3,
        "small": 0.5,
        "heavy": 0.2
    }
    # Conditional probability table (cpt) of varB knowing varA
    cpt_bl_i = {
        "absent": {
            "low": 0.6,
            "medium": 0.2,
            "high": 0.2
        },
        "small": {
            "low": 0.3,
            "medium": 0.3,
            "high": 0.4
        },
        "heavy": {
            "low": 0.1,
            "medium": 0.3,
            "high": 0.6
        }
    }
    cpt_w_i = {
        "absent": {
            1: 0.01,
            2: 0.09,
            3: 0.2,
            4: 0.3,
            5: 0.4
        },
        "small": {
            1: 0.2,
            2: 0.3,
            3: 0.3,
            4: 0.1,
            5: 0.1
        },
        "heavy": {
            1: 0.4,
            2: 0.3,
            3: 0.15,
            4: 0.1,
            5: 0.05
        }
    }
    cpt_FEV1_bl = {
        'low': {
            'low': 0.6,
            'medium': 0.3,
            'high': 0.2,
        },
        'medium': {
            'low': 0.3,
            'medium': 0.4,
            'high': 0.3,
        },
        'high': {
            'low': 0.1,
            'medium': 0.4,
            'high': 0.5,
        }
    }
    
    def sample(self):
        inflammation = utils.threeStatesSample(list(self.marginal_i.values()),
                                               list(self.marginal_i.keys()))
        
        wellness = utils.nStatesSample(list(self.cpt_w_i[inflammation].values()),
                                 list(self.cpt_w_i[inflammation].keys()))
        
        bacterial_load = utils.threeStatesSample(list(self.cpt_bl_i[inflammation].values()),
                                            list(self.cpt_bl_i[inflammation].keys()))
        
        FEV1 = utils.threeStatesSample(list(self.cpt_FEV1_bl[bacterial_load].values()),
                                list(self.cpt_FEV1_bl[bacterial_load].keys()))
        
        return {
            "Inflammation": inflammation,
            "Wellness": wellness,
            "Bacterial load": bacterial_load,
            "FEV1": FEV1
        }
model = lungModel()
model.sample()

{'Inflammation': 'small',
 'Wellness': 1,
 'Bacterial load': 'medium',
 'FEV1': 'low'}

In [5]:
# inference code
def observe(obs_B, key_value_pair_B):
    node_message_B = copy.copy(key_value_pair_B)
    for key, val in node_message_B.items():
        if key != obs_B: node_message_B[key] = 0
        else: node_message_B[key] = 1

    print("Observation:", node_message_B)
    return node_message_B

# factor node message
def message_down(dist):
    """
    dist: marginal distribution
    """
    print("Message down: ", dist)
    return dist

def factor_message_up(incoming_message, cpt_B_A):
    """
    The factor mode message is the product of all received messages on the other edges, multiplied by the conditional probability distribution for the factor and summed over all variables except the one being sent to
    :param incoming_message:
    :param cpt_B_A:
    :return:
    """
    # TODO: add a function to invert a conditional probability table to merge factor_message_up and factor_message_down
    # compute sum_B( P(B|A) )
    factor_message = utils.get_first_level_keys(cpt_B_A)
    for key_A, items_B in cpt_B_A.items():
        for key_B, p_B_A in items_B.items():
            factor_message[key_A] += p_B_A * incoming_message[key_B]

    message = utils.normalise(factor_message)
    print("Message up:", message)

    return message

def factor_message_down(incoming_message, cpt_B_A):
    """
    The factor mode message is the product of all received messages on the other edges, multiplied by the conditional probability distribution for the factor and summed over all variables except the one being sent to.
    :param incoming_message:
    :param cpt_B_A:
    :return:
    """
    # compute sum_B( P(B|A) )
    factor_message = utils.get_second_level_keys(cpt_B_A)
    for key_A, items_B in cpt_B_A.items():
        for key_B, p_B_A in items_B.items():
            factor_message[key_B] += p_B_A * incoming_message[key_A]

    message = utils.normalise(factor_message)
    print("Message down:", message)

    return message

def node_message(messages):
    """
    The variable node message is the product of all received messages on the other edges
    :param messages: incoming variable node messages
    :return: variable node belief message
    """
    # All messages have the same keys
    belief = utils.get_first_level_keys(messages[0], init_value=1)

    for key in belief.keys():
        for message in messages:
            belief[key] *= message[key]

    # Normalise
    p_A = utils.normalise(belief)
    
    print('Node belief:', p_A)
    return p_A


In [6]:
# define model
m = lungModel()

# CASE 1: observe bacterial load and infer inflammation
obs_bl = 'medium'

point_mass_bl = observe(obs_bl, utils.get_second_level_keys(m.cpt_bl_i))
up_bl = factor_message_up(point_mass_bl, m.cpt_bl_i)

down_i = message_down(m.marginal_i)

belief = node_message([up_bl, down_i])

Observation: {'low': 0, 'medium': 1, 'high': 0}
Message up: {'absent': 0.25, 'small': 0.37499999999999994, 'heavy': 0.37499999999999994}
Message down:  {'absent': 0.3, 'small': 0.5, 'heavy': 0.2}
Node belief: {'absent': 0.22222222222222224, 'small': 0.5555555555555556, 'heavy': 0.22222222222222224}


In [7]:
# CASE 2: observe bacterial load and wellness and infer inflammation
obs_fev1 = 'low'
obs_wellness = 3

point_mass_fev1 = observe(obs_fev1, utils.get_second_level_keys(m.cpt_FEV1_bl))
up_fev1 = factor_message_up(point_mass_fev1, m.cpt_FEV1_bl)
belief_bl = node_message([up_fev1]) # optional because there is only one message
up_bl = factor_message_up(up_fev1, m.cpt_bl_i)

point_mass_wellness = observe(obs_wellness, utils.get_second_level_keys(m.cpt_w_i))
up_wellness = factor_message_up(point_mass_wellness, m.cpt_w_i)

down_i = message_down(m.marginal_i)

belief = node_message([up_bl, up_wellness, down_i])

Observation: {'low': 1, 'medium': 0, 'high': 0}
Message up: {'low': 0.6000000000000001, 'medium': 0.30000000000000004, 'high': 0.10000000000000002}
Node belief: {'low': 0.6, 'medium': 0.3, 'high': 0.09999999999999999}
Message up: {'absent': 0.4583333333333333, 'small': 0.3229166666666667, 'heavy': 0.21874999999999997}
Observation: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0}
Message up: {'absent': 0.3076923076923077, 'small': 0.4615384615384615, 'heavy': 0.23076923076923075}
Message down:  {'absent': 0.3, 'small': 0.5, 'heavy': 0.2}
Node belief: {'absent': 0.3333333333333333, 'small': 0.5871212121212122, 'heavy': 0.07954545454545454}


In [9]:
# CASE 3: observe inflammation and infer bacterial load
point_mass_i = observe('small', m.marginal_i)
down_i = factor_message_down(point_mass_i, m.cpt_bl_i)

Observation: {'absent': 0, 'small': 1, 'heavy': 0}
Message down: {'low': 0.3, 'medium': 0.3, 'high': 0.4}


In [11]:
# CASE 4: observe wellness and infer fev1

obs_wellness = 1

point_mass_wellness = observe(obs_wellness, utils.get_second_level_keys(m.cpt_w_i))
up_wellness = factor_message_up(point_mass_wellness, m.cpt_w_i)

down_i = message_down(m.marginal_i)

belief_i = node_message([up_wellness, down_i])

down_bl = factor_message_down(belief_i, m.cpt_bl_i)
belief_bl = node_message([down_bl]) # optional because there is only one message

down_fev1 = factor_message_down(belief_bl, m.cpt_FEV1_bl)
belief_fev1 = node_message([down_fev1])


Observation: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0}
Message up: {'absent': 0.016393442622950817, 'small': 0.32786885245901637, 'heavy': 0.6557377049180327}
Message down:  {'absent': 0.3, 'small': 0.5, 'heavy': 0.2}
Node belief: {'absent': 0.016393442622950817, 'small': 0.5464480874316939, 'heavy': 0.43715846994535523}
Message down: {'low': 0.2174863387978142, 'medium': 0.29836065573770487, 'high': 0.48415300546448087}
Node belief: {'low': 0.2174863387978142, 'medium': 0.29836065573770487, 'high': 0.48415300546448087}
Message down: {'low': 0.26270189325061505, 'medium': 0.37020002139266234, 'high': 0.3670980853567226}
Node belief: {'low': 0.26270189325061505, 'medium': 0.37020002139266234, 'high': 0.3670980853567226}


## Gibbs sampling <a class="anchor" id="3.3"></a>

In [ ]:
# Gibbs inference code
def gibbs_inference_code(target_X, target_Y, modelToRun=lungModel, n=100000):


    # Random initialisation
    sample = modelToRun()
    i = [sample[target_X]]
    b = [sample[target_Y]]
    print('Initial state:', I, '=', i, ',', B, '=', b)

    # Loop
    # TODO  review function arguments to avoid global variables
    for j in np.arange(n):
        b.append(threeStatesSample(bacterial_load_p.get(i[j]), ["low", "medium", "high"]))
        i_posteriors = compute_posterior(I, B, b[j+1], conditional_probability_table, disp=0)
        i.append(threeStatesSample([i_posteriors[key] for key in i_posteriors.keys()], list(i_posteriors.keys())))

    return b, i

# Data processing
def get_marginal(name, list):
    '''
    name: identifier
    list: a list of values
    output: distinct values count in list, and related proportions (probability estimate)
    '''
    marginal = pd.DataFrame(data={name: list})
    marginal = pd.DataFrame(data={'count': marginal.value_counts()}).reset_index()
    marginal['P(count)'] = marginal['count']/marginal['count'].sum()
    
    return marginal

In [ ]:
# Gibbs sampling implementation for two variables

# Variables
I = 'Inflammation'
B = 'Bacterial load'
    
n_high=100000; n_low=1000
b, i = gibbs_inference_code(I, B, n=n_high)
b_fuzzy, i_fuzzy = gibbs_inference_code(I, B, n=n_low)

# Marginal distribution of inflammation
i_marginal = get_marginal(I, i)
i_marginal_fuzzy = get_marginal(I, i_fuzzy)
i_marginal

In [ ]:
# Marginal distribution of bacterial load
b_marginal = get_marginal(B, b)
b_marginal

In [ ]:
plotFigure(i_marginal, I, column=1, title="Marginal distribution for inflammation for "+str(n_high)+" iterations")
plotFigure(i_marginal_fuzzy, I, column=1, title="Marginal distribution for inflammation "+str(n_low)+" iterations")
plotFigure(b_marginal, B, column=1, title="Marginal distribution for inflammation for "+str(n_high)+" iterations")